# BlazingSQL + cuML NYC Taxi Cab Fare Prediction

This demo uses pubically availible [NYC Taxi Cab Data](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction) to predict the total fare of a taxi ride in New York City given the pickup and dropoff locations. 

In this notebook, we will cover: 
- How to read and query csv files with cuDF and BlazingSQL.
- How to implement a linear regression model with cuML.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=taxi_fare_prediction&dt=taxi_fare_prediction)


## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [2]:
# Import RAPIDS AI stack
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

lo
BlazingContext ready


### Download Data
For this demo we will train our model with 20,000,000 rows of data from 4 CSV files (5,000,000 rows each). 

The cell below will download them from AWS to the main `notebooks-contrib/data/blazingsql/` folder for you.

In [2]:
!wget -P ../../../data/blazingsql https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_00.csv
!wget -P ../../../data/blazingsql https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_01.csv
!wget -P ../../../data/blazingsql https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_02.csv
!wget -P ../../../data/blazingsql https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_03.csv
    
# tag relative path to data directory
path = '../../../data/blazingsql/'

## Extract, transform, load
In order to train our Linear Regression model, we must first preform ETL so to prepare our data.

### ETL: Read and Join CSVs

In [4]:
# set column names and types
col_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
col_types = ['date64', 'float32', 'float32', 'float32', 
                'float32', 'float32', 'float32']

# load first csv 
gdf_00 = cudf.read_csv(path+'taxi_00.csv', names=col_names, dtype=col_types)
# load second csv
gdf_01 = cudf.read_csv(path+'taxi_01.csv', names=col_names, dtype=col_types)
# load third csv
gdf_02 = cudf.read_csv(path+'taxi_01.csv', names=col_names, dtype=col_types)
# load fourth csv
gdf_03 = cudf.read_csv(path+'taxi_01.csv', names=col_names, dtype=col_types)

# combine all those dataframes into one master dataframe
gdf = cudf.concat([gdf_00,gdf_01, gdf_02, gdf_03])

# what's it look like?
gdf.head()

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-02-02 22:30:19.002,8.9,-73.988708,40.758804,-73.986519,40.737202,1.0
1,2014-09-20 07:19:24.001,4.0,-73.990204,40.746708,-73.994728,40.750515,1.0
2,2013-02-23 07:18:05.001,5.5,-74.016762,40.709438,-74.009003,40.719498,3.0
3,2015-04-18 23:49:27.009,13.5,-74.002708,40.733730,-73.986099,40.734776,1.0
4,2010-03-04 08:15:59.001,10.5,-73.988365,40.737663,-74.012459,40.713932,1.0


### ETL: Create Table

In [5]:
%time
# make a table from the combined df
bc.create_table('taxi', gdf, column_names=column_names)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


### ETL: Query Tables for Training Data

In [8]:
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            taxi
        '''

# run query on table (returns cuDF DataFrame)
X_train = bc.sql(query)

# fill null values 
X_train['longitude_distance'] = X_train['longitude_distance'].fillna(0)
X_train['latitude_distance'] = X_train['latitude_distance'].fillna(0)
X_train['passenger_count'] = X_train['passenger_count'].fillna(0)

# how's it look? 
X_train.head()

20196


,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,22.0,2.0,12.0,0.002190,-0.021603,1.0
1,7.0,9.0,14.0,-0.004524,0.003807,1.0
2,7.0,2.0,13.0,0.007759,0.010059,3.0
3,23.0,4.0,15.0,0.016609,0.001045,1.0
4,8.0,3.0,10.0,-0.024094,-0.023731,1.0


In [14]:
# query dependent variable y
y_train = bc.sql('SELECT fare_amount FROM main.taxi')

y_train.head()

20196


,fare_amount
0,8.9
1,4.0
2,5.5
3,13.5
4,10.5


## Linear Regression
### LR: Train Model

In [15]:
from cuml import LinearRegression

%%time
# call & create cuML model
lr = LinearRegression(fit_intercept=True, normalize=False, algorithm="eig")

# train Linear Regression model 
reg = lr.fit(X_train, y_train)

# display results
print(f"Coefficients:\n{reg.coef_}\n")
print(f"Y intercept:\n{reg.intercept_}\n")

ModuleNotFoundError: No module named 'cuml'

### LR: Use Model to Predict Future Taxi Fares 

Test data for this notebook is already stored in `notebooks-contrib/data/blazingsql/` so there's no need to download it. We are, however, going to create this table directly from CSV, and BlazingSQL requires the full path to the data for table creation. 

This cell uses the `pwd` bash command to identify the path to this directory, then joins it with the relative path to the notebooks-contrib `data/` directory to provide a full path to the test data.

In [2]:
# identify path to this notebook, !pwd returns SList w/ path (str) at 0th index
working_directory = !pwd
# extract path to this notebook & add path from here to data
path = working_directory[0] + path

# how's it look?
path

'/Users/winston/temp/notebooks-contrib/intermediate_notebooks/examples/blazingsql../../../data/blazingsql/'

In [19]:
# set column names and types
col_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
col_types = ['date64', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32']

# create test table directly from CSV
bc.create_table('test', path, names=col_names, dtype=col_types)

In [20]:
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            test
        '''

# run query on table (returns cuDF DataFrame)
X_test = bc.sql(query)

# fill null values 
X_test['longitude_distance'] = X_test['longitude_distance'].fillna(0)
X_test['latitude_distance'] = X_test['latitude_distance'].fillna(0)
X_test['passenger_count'] = X_test['passenger_count'].fillna(0)

# how's it look? 
X_test.head()

20196


,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,13.0,1.0,15.0,-0.008110,-0.019970,1.0
1,13.0,1.0,15.0,-0.012024,0.019814,1.0
2,11.0,10.0,11.0,0.002869,-0.005119,1.0
3,21.0,12.0,12.0,-0.009277,-0.016178,1.0
4,21.0,12.0,12.0,-0.022537,-0.045345,1.0


In [21]:
# predict fares 
predictions = lr.predict(X_test_gdf)

# display predictions
predictions

NameError: name 'lr' is not defined

In [19]:
# combine into a table of table points and predictions
X_test['predicted_fare'] = predictions

# how's that look?
X_test.head()

,hours,months,years,longitude_distance,latitude_distance,passenger_count,predicted_fare
0,13.0,1.0,15.0,-0.008110,-0.019970,1.0,12.778599
1,13.0,1.0,15.0,-0.012024,0.019814,1.0,12.778547
2,11.0,10.0,11.0,0.002869,-0.005119,1.0,11.284673
3,21.0,12.0,12.0,-0.009277,-0.016178,1.0,11.864964
4,21.0,12.0,12.0,-0.022537,-0.045345,1.0,11.864986


## Real Life Example  
Predict cost of a ride from Grand Central Station to Samsung Next NYC at 7:00 AM on May 15th, 2020.

In [20]:
# build a sample dataframe with cuDF
samsung_ride = cudf.DataFrame([('hours', 7.0), ('months', 5.0), 
                               ('years', 20.0), ('longitude_distance', 0.012727), 
                               ('latitude_distance', 0.008484), ('passenger_count', 1.0)])

# make prediction
samsung_prediction = lr.predict(samsung_ride)

# output fare prediction
samsung_prediction

0    16.517778
dtype: float32